# Exercise 3

## Importierung der Programmbibliotheken

Die ```json``` Programmbibliothek ermöglicht das Nutzen von Funktionen, die das Bearbeiten von .json Dateien erleichtert.  
Die ```requests``` Programmbibliothek erlaubt das einfache Ausgeben des HTML-Codes von Webseiten.  
Die ```csv``` Programmbibliothek ermöglicht das Nutzen von Funktionen, die das Bearbeiten von CSV Dateien erleichtert.    
Die ```re```(regex) Programmbibliothek ermöglicht das komplexe Suchen von Strings.  
Die Programmbibliothek ```pprint``` hilft, komplette Datenstrukturen übersichtlicher darzustellen.
Die Programmbibliothek ```BeatifulSoup``` hilft unter anderem, HTML und XML-Codes zu lesen und zu strukturieren.  

In [1]:
import json
import requests
import csv
import re
import pprint
from bs4 import BeautifulSoup

## Definition der Funktionen

### Funktion 1 ( "Suche" )

Die Funktion ```Suche``` ist dafür da, den vorsortierten HTML Code nach für uns nützlichen Informationen zu durchsuchen.  
Die Übergabeparameter ```Suche1,Suche2,Suche3``` sind RegEx-Angaben zur Verfeinerung und Informationsgewinnung aus dem HTML Code.  
```infotable``` ist ein Übergabeparameter, der den vor verfeinerten HTML Code übergibt.  
```nan``` ist die Bezeichnung, falls die RegEx-Suche nichts ergibt. Der Parameter muss übergeben werden, da die ```strings``` teilsweise noch bearbeitet werden.


Die ersten drei Anweisungen der Funktion durchsucht einen ```string``` nach einer RegEx-Anweisung und speichert dies in eine neue Variable. Die ```if``` Abfrage kontrolliert, ob das Ergebnis der letzten Suche eine leere Liste ist. Ist dies der Fall, hat die Suche keine Ergebnisse. Dadurch wird die ```nan``` Variable angehangen.  
Der ```return``` Value der Funktion ist ```info```. Dies ist eine Liste der Ergebnisse oder der Listeneintrag ```nan``` ist nicht vorhanden.

In [2]:
def Suche(Suche1,Suche2,Suche3,infotable,nan):
    suchergeb1 = re.findall(Suche1,str(infotable))
    suchergeb2 = re.findall(Suche2,str(suchergeb1))
    info = re.findall(Suche3,str(suchergeb2))  
    
    if info == []:
        
        info = [nan]
        
    return info

### Funktion 2 ( "writecsv" )

Die Funktion ```writeCSV``` zum Schreiben einer CSV Datei.  
Der Übergabe Parameter ```namecountsschar``` ist ein String, dessen Inhalt der gewünschte Name ist.  
Beim Übergabeparameter ```Dic``` wird ein Dictionary übergeben, das in eine CSV Datei gespeichert werden soll.  
In der ersten Anweisung der Funktion wird ```fieldsdialog``` definiert und je nach Benennung verändern sich die Feldnamen der CSV Datei.  
Die ```with open``` Funktion öffnet Dateien, solange sie in ihrer Syntax sind.  
Als Parameter werden das ```w``` übergeben, welches für ```write``` steht und die geöffnete Datei zu schreiben ermöglicht und ```newline=""```, welches bestimmt, was nach einem Absatz passiert.  
Durch den Ausdruck wird ```writer``` ein csv.writer Objekt und übergibt die oben genannten Parameter an.
```writer.writerow``` Hier wird die erste Zeile definiert. (Mit den Feldnamen die oben definiert wurden)
```writer.writerows```(zip(Dic.keys(),Dic.values())) Diese Anweisung schreibt die Werte des Dictionary in die csv Datei.

In [3]:
def writecsv(namecountsschar, Dic):
    
    
    
    
    fieldsdialog = ['Länder','Links']
    
    
    with open(namecountsschar, "w", newline="") as csvfile:
        
        writer = csv.writer(csvfile)
        
        writer.writerow(fieldsdialog)
        
        writer.writerows(zip(Dic.keys(),Dic.values()))

## URLs der Tabelle

```jsonFilePath``` ist der Name der ```.json``` Datei, indem das Ergebnis gespeichert werden soll.  
Die Variable ```website_url``` ist das Ergebnis einer Anfrage mithilfe der ```requests.get``` Funktion, welches den Quellcode der Wikipedia Seite "List of countries by meat consumption" abfragt. 
```.text``` sorgt dafür, dass es in einen ```string``` umgewandelt wird und es gelesen werden kann.  
Mithilfe der ```BeatifulSoup``` Funktion wird der ```string``` in einen ```BeautifulSoup``` umgewandelt, damit der Code der Tabelle herausgefiltert werden kann.  
Danach wird der Code der Tabelle nach internen Wikipedia-Links durchsucht. Die Schleife geht die Liste der internen Wikipedialinks durch und liest diese nach und nach in eine ```dictionary``` ein bestehend aus den Ländernamen und der gesamten URL.  
Die URL setzt sich zusammen aus der Wikipedia Seite: "https://www.wikipedia.org/" und dem internen Link abzüglich der letzten beiden Leerzeichen.

In [4]:
jsonFilePath = "exercise3.json"

website_url = requests.get("https://en.wikipedia.org/wiki/List_of_countries_by_meat_consumption").text

soup = BeautifulSoup(website_url,"html.parser")

countries = soup.find("table",{"class":"wikitable"})

b = re.findall(r"/wiki/.*\" ",str(countries))

Dic = {}

for i in b:
    
    Dic[i[6:-2]] = "https://en.wikipedia.org" + i[:-2]

writecsv("Listelinks.csv",Dic)

## Infobox Informationen zu JSON

Die Schleife geht das ```dictionary``` Stück für Stück durch und gibt den ```key``` Attribut aus. Mithilfe des ```key``` Attributs wird aus dem ```dictionary``` die URL aufgerufen und die zwei darauf folgenden Schritte wie oben wiederholt.  
Im nächsten Schritt wird die bereits oben definierte Funktion ```Suche``` genutzt, um die Größe der Länder zu extrahieren. Die erste RegEx-Funktion extrahiert den gesamten Quellcode zwischen dem Auftauchen von "Area" und dem Ort, an dem die Informationen gespeichert sind.  
Die Extraktion der Hauptstadt ist ein Sonderfall, da diese nur zwei Schritte benötigt. Deswegen kann die Funktion ohne weiteres genutzt werden.  
Die Bevölkerungszahl und die Bevölkerung werden wie oben extrahiert.  
Das ```dictionary```, welches die Ergebnisse beinhaltet, hat als ```key``` Attribut den Ländernamen und als ```value``` Attribut weitere dictionaries mit den vorher extrahierten Informationen und deren Bezeichnungen.

In [5]:
for i in Dic:
    
    Captial = " "
    Area = " "
    Demonym = " "
    Pop = " "
    
    website_url = requests.get(Dic[i]).text
    soup = BeautifulSoup(website_url,"html.parser")
    infotable =  soup.find("table", class_="infobox geography vcard")
    
     
    Area = Suche(r"Area.+?(?=</td>)",r"</th><td>(.+?<sup)",r"[0-9,–]{2,12}",infotable,"nan")
       
    c = re.findall(r"Capital.+?(?=</a>)",str(infotable))
    Captial = re.findall(r"title=\"(.+?)\"",str(c))
    
    if Captial == []:
        
        Captial = ["nan"]
    
    Pop = Suche(r"Population.+?(?=</a>)",r"</th><td>(.+?<sup)",r"[0-9,]{2,15}<sup|[0-9,]{2,15} <sup",infotable,"nan1234")
    
    
    Demonym = Suche(r"Demonym.+?(?=</td>)",r"title=\"[^\"Demonym\"](.{1,40}a>)",r"\"(.+?)<\/a",infotable,"1nan")           
    
    
    Dic[i] = {
              "Capital City" : str(Captial[0]),
              "Area" : str(Area[0]) +"km^2",
              "Demonym" : str(Demonym[0])[1:],
              "Population" : str(Pop[0])[:-4]
              }
    
    
    
    
    
    
with open(jsonFilePath, 'w') as jsonFile:
    jsonFile.write(json.dumps(Dic,indent=4))    
    
# pprint.pprint(Dic) 

KeyboardInterrupt: 